In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from alive_progress import alive_bar
import time
import pandas as pd

In [7]:
s = Service(ChromeDriverManager(path=r".\\").install())
driver = webdriver.Chrome(service=s)
print("-> Launching Chrome...")
driver.get("https://bbc.com/news/world-60525350")

urls = []

# Ignored Exceptions
ignored_exceptions = (
    NoSuchElementException,
    StaleElementReferenceException,
)

# Waits
wait = WebDriverWait(driver, 10)
wait2 = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions)
wait3 = WebDriverWait(driver, 5)

buffering_time = 1

# CSS Selector Tags
popup_tag = "button[class='tp-close tp-active']"
title_tag = "a[class='qa-heading-link lx-stream-post__header-link']"
nextPage_tag = "a[class='lx-pagination__btn gs-u-mr+ qa-pagination-next-page lx-pagination__btn--active']"
numPages_tag = (
    "span[class='lx-pagination__page-number qa-pagination-total-page-number']"
)

# Scraper
try:
    print("-> Awaiting pop-ups...", end=" ")
    popup = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, popup_tag)))
finally:
    popup.click()
    print("Pop-up eliminated.")
num_pages = 2  # int(driver.find_element(By.CSS_SELECTOR, numPages_tag).text)

with alive_bar(num_pages, spinner="dots_waves", bar="smooth", force_tty=True) as bar:
    for page in range(num_pages - 1):
        bar()
        time.sleep(buffering_time)
        titles = wait2.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, title_tag))
        )
        for title in titles:
            urls.append(title.get_attribute("href"))

        next = driver.find_element(By.CSS_SELECTOR, nextPage_tag)
        next.click()
    bar()

print(f"-> {len(urls)} URLs fetched successfully!")
driver.quit()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [.\\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


-> Launching Chrome...
-> Awaiting pop-ups... Pop-up eliminated.
|████████████████████████████████████████| 2/2 [100%] in 2.6s (0.77/s)                                                  
-> 20 URLs fetched successfully!


In [11]:
s = Service(ChromeDriverManager(path=r".\\").install())
driver = webdriver.Chrome(service=s)
print("-> Launching Chrome...")

texts = []
titles = []
index = 0

pgTitle_tag = "h1[class='ssrcss-15xko80-StyledHeading e1fj1fc10']"
pgTitle_tag_2 = "h1[class='ssrcss-1qr3f1s-StyledHeading e1fj1fc10']"
pgText_tag = "p[class='ssrcss-1q0x1qg-Paragraph eq5iqo00']"


def check_exists(tag):
    try:
        driver.find_element(By.CSS_SELECTOR, tag)
    except NoSuchElementException:
        return False
    return True


for url in urls:

    tempTitle = ""
    tempText = ""

    print(url)
    driver.get(url)

    print("-> Awaiting pop-ups...", end=" ")
    if check_exists(popup_tag):
        popup = driver.find_element(By.CSS_SELECTOR, popup_tag)
        popup.click()
        print("Pop-up eliminated.")
    else:
        print("No pop-ups found.")

    if check_exists(pgTitle_tag):
        tempTitle += driver.find_element(By.CSS_SELECTOR, pgTitle_tag).text
    elif check_exists(pgTitle_tag_2):
        tempTitle += driver.find_element(By.CSS_SELECTOR, pgTitle_tag_2).text
    print(f"-> Title: \"{tempTitle}\"")

    if check_exists(pgText_tag):
        phrases = driver.find_elements(By.CSS_SELECTOR, pgText_tag)
        for phrase in phrases:
            tempText += " " + phrase.text
    print(f"-> Word count: {len(tempText.split())}")

    texts.append(tempText)
    titles.append(tempTitle)

print("Database generated successfully!")
driver.quit()
dict_data = {"Title": titles, "Text": texts}
data = pd.DataFrame(dict_data)
data.head(5)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [.\\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


-> Launching Chrome...
https://www.bbc.com/news/world-61574412
-> Awaiting pop-ups... No pop-ups found.
-> Title: "Ros Atkins on... Russia's food war"
-> Word count: 110
https://www.bbc.com/news/business-61569560
-> Awaiting pop-ups... Pop-up eliminated.
-> Title: "US closes loophole for Russian debt payments"
-> Word count: 477
https://www.bbc.com/news/world-61570444
-> Awaiting pop-ups... No pop-ups found.
-> Title: "Ukraine war: 'This is just the beginning, everything is still to come'"
-> Word count: 877
https://www.bbc.com/news/world-europe-61569867
-> Awaiting pop-ups... No pop-ups found.
-> Title: "Ukraine war round-up: Fighting in the east and a seaside home for refugees"
-> Word count: 751
https://www.bbc.com/news/world-europe-61568999
-> Awaiting pop-ups... No pop-ups found.
-> Title: "Ukraine war: Put values over profits, Nato chief tells countries"
-> Word count: 435
https://www.bbc.com/news/world-asia-61560784
-> Awaiting pop-ups... No pop-ups found.
-> Title: "Ukraine war

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0054B8F3+2406643]
	Ordinal0 [0x004DAF31+1945393]
	Ordinal0 [0x003CC748+837448]
	Ordinal0 [0x003B42B0+737968]
	Ordinal0 [0x00416C09+1141769]
	Ordinal0 [0x00423E22+1195554]
	Ordinal0 [0x00414096+1130646]
	Ordinal0 [0x003EE636+976438]
	Ordinal0 [0x003EF546+980294]
	GetHandleVerifier [0x007B9612+2498066]
	GetHandleVerifier [0x007AC920+2445600]
	GetHandleVerifier [0x005E4F2A+579370]
	GetHandleVerifier [0x005E3D36+574774]
	Ordinal0 [0x004E1C0B+1973259]
	Ordinal0 [0x004E6688+1992328]
	Ordinal0 [0x004E6775+1992565]
	Ordinal0 [0x004EF8D1+2029777]
	BaseThreadInitThunk [0x76D7FA29+25]
	RtlGetAppContainerNamedObjectPath [0x772D7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x772D7A4E+238]
